# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
df = pd.read_csv("data/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Baseline_points_2',
       'Baseline_points_30', 'Baseline_points_4', 'Bonus_2', 'Bonus_30',
       'Bonus_4', 'CrdR_2', 'CrdR_30', 'CrdR_4', 'CrdY_2', 'CrdY_30', 'CrdY_4',
       'DEF', 'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GW', 'Gls_2', 'Gls_30', 'Gls_4', 'ICT_index_2',
       'ICT_index_30', 'ICT_index_4', 'MID', 'Min_2', 'Min_30', 'Min_4',
       'Name_original', 'Opp_rating', 'Opp_score_2', 'Opp_score_30',
       'Opp_score_4', 'Opponent', 'Rating_difference', 'Season', 'Sh_2',
       'Sh_30', 'Sh_4', 'SoT_2', 'SoT_30', 'SoT_4', 'Start_2', 'Start_30',
       'Start_4', 'Team', 'Team_CS_2', 'Team_CS_30', 'Team_CS_4',
       'Team_rating', 'Team_result_2', 'Team_result_30', 'Team_result_4',
       'Team_score_2', 'Team_score_30', 'Team_score_4', 'Was_home', 'xA_2',
       'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'Finished', 'Date'],
      dtype='object')

In [5]:
rolling_gameweeks = [2, 4, 30]
features_to_roll = [
                "Min", "Start", # time played
                'Gls', 'Sh', 'SoT', # Goals
                'Ast', # Assists
                'CrdY', 'CrdR', # Cards
                "xG", 'xA', # Expected
                'Team_CS', # Defence
                'Team_score', 'Opp_score', 'Team_result', # Team form
                # "xGPoints", "CSPoints", # Position-scaled
                # "Cmp%", "PrgP", "PrgC", "T_succ",
                # 'bonus', 'bps', # Bonus
                # 'Influence', 'Creativity', 'Threat', 
                'ICT_index', # ICT
                "FPL_points", "Baseline_points", "Bonus" # FPL points
            ]
features = ["Was_home",
            "Team_rating", "Opp_rating", "Rating_difference", 
            "DEF", "MID", "FWD"]

for r in rolling_gameweeks:
    features += [f'{col}{"_"}{r}' for col in features_to_roll]
    
features = ['Was_home',
 'Team_rating',
 'Opp_rating',
 'Rating_difference',
 'Min_2',
 'Start_2',
 'Gls_2',
 'Sh_2',
 'SoT_2',
 'Ast_2',
 'CrdY_2',
 'CrdR_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'ICT_index_2',
 'FPL_points_2',
 'Baseline_points_2',
 'Bonus_2',
 'Min_4',
 'Start_4',
 'Gls_4',
 'Sh_4',
 'SoT_4',
 'Ast_4',
 'CrdY_4',
 'CrdR_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'ICT_index_4',
 'FPL_points_4',
 'Baseline_points_4',
 'Bonus_4',
 'Min_30',
 'Start_30',
 'Gls_30',
 'Sh_30',
 'SoT_30',
 'Ast_30',
 'CrdY_30',
 'CrdR_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'ICT_index_30',
 'FPL_points_30',
 'Baseline_points_30',
 'Bonus_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos"]
to_predict = ["FPL_points"]

In [6]:
len(features)

61

In [7]:
# df = df[np.unique(features + info)]

In [8]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [9]:
model

GradientBoostingRegressor(random_state=42)

In [10]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    
    preds = df_predictions[info + ["Pred"]].sort_values(by=["Pred"], ascending = False)
    
    return preds, predictions

In [11]:
# df

In [12]:
X_test = df[df["Season"] == CURRENT_SEASON][features]
# .reset_index(drop=True)

In [13]:
X_test.columns

Index(['Was_home', 'Team_rating', 'Opp_rating', 'Rating_difference', 'Min_2',
       'Start_2', 'Gls_2', 'Sh_2', 'SoT_2', 'Ast_2', 'CrdY_2', 'CrdR_2',
       'xG_2', 'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2',
       'Team_result_2', 'ICT_index_2', 'FPL_points_2', 'Baseline_points_2',
       'Bonus_2', 'Min_4', 'Start_4', 'Gls_4', 'Sh_4', 'SoT_4', 'Ast_4',
       'CrdY_4', 'CrdR_4', 'xG_4', 'xA_4', 'Team_CS_4', 'Team_score_4',
       'Opp_score_4', 'Team_result_4', 'ICT_index_4', 'FPL_points_4',
       'Baseline_points_4', 'Bonus_4', 'Min_30', 'Start_30', 'Gls_30', 'Sh_30',
       'SoT_30', 'Ast_30', 'CrdY_30', 'CrdR_30', 'xG_30', 'xA_30',
       'Team_CS_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30',
       'ICT_index_30', 'FPL_points_30', 'Baseline_points_30', 'Bonus_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [14]:
preds, predictions = get_predictions(model, df, X_test)

15761 (15761, 71)


In [15]:
# next gameweek
preds[preds["GW"] == 1].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Pred
4591,Erling Haaland,1.0,2023-24,Manchester City,Burnley,0.0,2077.251953,1726.401489,FWD,9.237093
10005,Marcus Rashford,1.0,2023-24,Manchester Utd,Wolves,1.0,1873.631104,1719.037964,MID,8.721724
5120,Gabriel Martinelli Silva,1.0,2023-24,Arsenal,Nott'ham Forest,1.0,1919.368774,1673.962646,MID,8.449322
5082,Gabriel Fernando de Jesus,1.0,2023-24,Arsenal,Nott'ham Forest,1.0,1919.368774,1673.962646,FWD,8.290721
10725,Michael Olise,1.0,2023-24,Crystal Palace,Sheffield United,1.0,1757.395630,1644.073120,MID,8.272587
13905,Sean Longstaff,1.0,2023-24,Newcastle Utd,Aston Villa,1.0,1875.999878,1827.994629,MID,8.194623
10119,Martin Ødegaard,1.0,2023-24,Arsenal,Nott'ham Forest,1.0,1919.368774,1673.962646,MID,8.138465
15496,Willian Borges da Silva,1.0,2023-24,Fulham,Everton,0.0,1736.636841,1708.234497,MID,7.738151
7623,Jordan Ayew,1.0,2023-24,Crystal Palace,Sheffield United,1.0,1757.395630,1644.073120,MID,7.427771
911,Anthony Martial,1.0,2023-24,Manchester Utd,Wolves,1.0,1873.631104,1719.037964,FWD,7.301323


In [16]:
# all preds
preds.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Pred
10029,Marcus Rashford,25.0,2023-24,Manchester Utd,Luton,1.0,1873.631104,1606.522583,MID,9.754518
10016,Marcus Rashford,12.0,2023-24,Manchester Utd,Luton,1.0,1873.631104,1606.522583,MID,9.754518
4591,Erling Haaland,1.0,2023-24,Manchester City,Burnley,0.0,2077.251953,1726.401489,FWD,9.237093
4611,Erling Haaland,22.0,2023-24,Manchester City,Burnley,0.0,2077.251953,1726.401489,FWD,9.237093
4594,Erling Haaland,4.0,2023-24,Manchester City,Fulham,0.0,2077.251953,1736.636841,FWD,9.237093
4607,Erling Haaland,17.0,2023-24,Manchester City,Crystal Palace,0.0,2077.251953,1757.395630,FWD,9.237093
4608,Erling Haaland,19.0,2023-24,Manchester City,Everton,0.0,2077.251953,1708.234497,FWD,9.237093
4597,Erling Haaland,7.0,2023-24,Manchester City,Wolves,0.0,2077.251953,1719.037964,FWD,9.237093
4613,Erling Haaland,24.0,2023-24,Manchester City,Everton,0.0,2077.251953,1708.234497,FWD,9.237093
4626,Erling Haaland,37.0,2023-24,Manchester City,Fulham,0.0,2077.251953,1736.636841,FWD,9.237093


In [17]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Erling Haaland,338.614074
Marcus Rashford,300.398725
Gabriel Martinelli Silva,299.888508
Martin Ødegaard,292.384174
Gabriel Fernando de Jesus,285.648877
Sean Longstaff,283.443893
Bukayo Saka,246.790147
Mohamed Salah,241.184583
Julián Álvarez,235.152202


In [18]:
preds.to_csv("predictions/preds_next_season.csv", index=False)

In [19]:
preds.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Pred
10029,Marcus Rashford,25.0,2023-24,Manchester Utd,Luton,1.0,1873.631104,1606.522583,MID,9.754518
10016,Marcus Rashford,12.0,2023-24,Manchester Utd,Luton,1.0,1873.631104,1606.522583,MID,9.754518
4591,Erling Haaland,1.0,2023-24,Manchester City,Burnley,0.0,2077.251953,1726.401489,FWD,9.237093
4611,Erling Haaland,22.0,2023-24,Manchester City,Burnley,0.0,2077.251953,1726.401489,FWD,9.237093
4594,Erling Haaland,4.0,2023-24,Manchester City,Fulham,0.0,2077.251953,1736.636841,FWD,9.237093
